In [7]:
import pandas as pd
# pd.options.display.max_rows = 20
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from IPython.display import clear_output
from itertools import islice

In [23]:
import pandas as pd
datasets_location = "./datasets/"
chunk_size = 20000  # Adjust the chunk size as needed
raw_chunks = pd.read_csv(datasets_location + 'animelist.csv', chunksize=chunk_size)
chunk_count = number_of_chunks = sum([1 for _ in islice(raw_chunks, None)])
print(chunk_count)
raw_chunks = pd.read_csv(datasets_location + 'animelist.csv', chunksize=chunk_size)

5462


In [10]:
animeDF = pd.read_csv(datasets_location + 'anime.csv')
columns = ["Genres"]
animeDFObj = {}
for chunkColumn in columns:
    animeDFObj["length"] = 0
    animeDFObj["columns"] = {}
    animeDFObj["columnsList"] = ['action', 'adventure', 'comedy', 'drama', 'sci-fi', 'space', 'mystery', 'shounen', 'police', 'supernatural', 'magic', 'fantasy', 'sports', 'josei', 'romance', 'slice of life', 'cars', 'seinen', 'horror', 'psychological', 'thriller', 'super power', 'martial arts', 'school', 'ecchi', 'vampire', 'military', 'historical', 'dementia', 'mecha', 'demons', 'samurai', 'game', 'shoujo', 'harem', 'music', 'shoujo ai', 'shounen ai', 'kids', 'hentai', 'parody', 'yuri', 'yaoi']

    def Genres(row, animeDFObj, isAdd):
        text = row[chunkColumn]
        # print(text)
        text = text.split(",")
        if isAdd:
            animeDFObj["length"] += len(text)
        if isAdd:
            for column in text:
                column = column.lower().strip()
                if column == "unknown": 
                    continue
                # print(animeDFObj["columns"][column])
                if column in animeDFObj["columns"].keys():
                    animeDFObj["columns"][column] +=  1
                else:
                    animeDFObj["columns"][column] = 1
        else:
            for column in text:    
                column = column.lower().strip()
                if column == "unknown": 
                    continue
                # row[column] = round(animeDFObj["columns"][column] / animeDFObj["length"],3)
                row[column] = 1
        return row

    animeDF.apply(lambda x: Genres(x, animeDFObj, True),axis = 1)

    animeDFObj["columnsList"] = animeDFObj["columns"].keys()

    for column in animeDFObj["columnsList"]:
        if column == "unknown": 
            continue
        animeDF[column] = np.zeros((animeDF.shape[0], 1), dtype=int)


    animeDF = animeDF.apply(lambda x: Genres(x,animeDFObj,False),axis = 1)

In [11]:
pd.read_csv(datasets_location + 'watching_status.csv')

,status,description
0,1,Currently Watching
1,2,Completed
2,3,On Hold
3,4,Dropped
4,6,Plan to Watch


In [12]:
watching_status = pd.read_csv(datasets_location + 'watching_status.csv')
watching_statusObj = {}
for i in watching_status[["status"," description"]].values:
    # print(i)
    watching_statusObj[i[0]] = i[1]
watching_statusObj[5] = "unknown"
print(watching_statusObj)

{1: 'Currently Watching', 2: 'Completed', 3: 'On Hold', 4: 'Dropped', 6: 'Plan to Watch', 5: 'unknown'}


In [13]:
def calculateRemaining(primary,isInc):
    remaining = 1 - primary if primary >= 0.5 else primary
    
    if round(primary,3) == 1:
        remaining = 0.5 if isInc == 0 else 0
        
    if round(primary,3) == 0:
        remaining = 0 if isInc == 0 else 0.5
        
    if (isInc == 0.001 or isInc == 0):
        result = primary - (remaining / 2)
    else:
        result = primary + (remaining / 2)
    result = 0 if result < 0 else result;
    # print(primary,isInc,"  ",remaining,result)
    return round(result, 4)

In [24]:
temp_count = 0
for chunk in raw_chunks:
    # print(chunk.shape)
    chunk = pd.concat([chunk, pd.DataFrame(np.zeros((chunk.shape[0],len(animeDFObj["columnsList"])), dtype=int),columns = animeDFObj["columnsList"])], axis=1)
    chunk["watching_status"].fillna(5, inplace=True)
    chunk["watching_status"] = chunk["watching_status"].apply(lambda x : watching_statusObj[x if x else 5])
    chunk = pd.get_dummies(chunk, columns=['watching_status'])
    
    # print(chunk)
    for userid,user in chunk.groupby("user_id"):
        columnsList = animeDFObj["columnsList"]
        selected_data = animeDF.loc[animeDF["MAL_ID"] == user["anime_id"].values[0], columnsList]
        # print(user[["user_id","anime_id"]])
        temp = False
        for index in range(0,user.shape[0]):
            #user.iloc[0][columnsList] = selected_data.values[0]
            anime_id = user.iloc[index]["anime_id"]
            # action = user.iloc[index]["action"]
            # print(userid,anime_id,action)
            if(index > 0):
                selected_data = animeDF.loc[animeDF["MAL_ID"] == anime_id, columnsList]
                selected_data = selected_data.apply(lambda x: [calculateRemaining(temp[x.name].values[0],x.values[0])],axis = 0)
                # print(temp.values,selected_data.values)
                # fig, ax = plt.subplots()
                # ax.plot(temp.values.reshape(-1), label='temp')
                # ax.plot(selected_data.values.reshape(-1), label='selected_data')
                # plt.legend()
                # plt.show()
                # clear_output(wait=True)
                
                temp = selected_data
            else:
                temp = animeDF.loc[animeDF["MAL_ID"] == anime_id, columnsList]
            chunk.loc[(chunk["user_id"] == userid) & (chunk["anime_id"] == anime_id), columnsList] = selected_data.values
    temp_count += 1
    print(round(temp_count/chunk_count))
    break

0


raw_chunks.to_csv(datasets_location + 'modifiedAnimeList.csv', index=False)

In [22]:
print(user.loc[user["anime_id"] == anime_id, columnsList].shape)
print(temp.values.reshape(1,-1).shape)

NameError: name 'user' is not defined

In [208]:
user[user["user_id"] == userid]

,user_id,anime_id,rating,watched_episodes,action,adventure,comedy,drama,sci-fi,space,...,hentai,parody,yuri,yaoi,watching_status_Completed,watching_status_Currently Watching,watching_status_Dropped,watching_status_On Hold,watching_status_Plan to Watch,watching_status_unknown
2220000,7468.0,186.0,9.0,13.0,1.0000,0.0000,0.0000,1.0000,0.0000,0.0,...,0.0000,0.0000,0.0,0.0,1,0,0,0,0,0
2220001,7468.0,187.0,9.0,1.0,1.0000,0.0000,0.0000,1.0000,0.0000,0.0,...,0.0000,0.0000,0.0,0.0,1,0,0,0,0,0
2220002,7468.0,21327.0,8.0,12.0,0.7500,0.0000,0.2500,0.7500,0.0000,0.0,...,0.0000,0.0000,0.0,0.0,1,0,0,0,0,0
2220003,7468.0,7054.0,10.0,26.0,0.6250,0.0000,0.3750,0.6250,0.0000,0.0,...,0.0000,0.0000,0.0,0.0,1,0,0,0,0,0
2220004,7468.0,9311.0,8.0,2.0,0.4375,0.0000,0.1875,0.4375,0.0000,0.0,...,0.2500,0.0000,0.0,0.0,1,0,0,0,0,0
2220005,7468.0,3102.0,8.0,2.0,0.2188,0.0000,0.0938,0.2188,0.0000,0.0,...,0.3750,0.0000,0.0,0.0,1,0,0,0,0,0
2220006,7468.0,17741.0,5.0,12.0,0.1094,0.0000,0.0469,0.3282,0.0000,0.0,...,0.1875,0.0000,0.0,0.0,1,0,0,0,0,0
2220007,7468.0,28851.0,10.0,1.0,0.0547,0.0000,0.0234,0.4923,0.0000,0.0,...,0.0938,0.0000,0.0,0.0,1,0,0,0,0,0
2220008,7468.0,11771.0,9.0,25.0,0.0274,0.0000,0.0351,0.2462,0.0000,0.0,...,0.0469,0.0000,0.0,0.0,1,0,0,0,0,0
2220009,7468.0,16894.0,10.0,25.0,0.0137,0.0000,0.0526,0.1231,0.0000,0.0,...,0.0234,0.0000,0.0,0.0,1,0,0,0,0,0


In [213]:
chunk[chunk["user_id"] == userid] 

,user_id,anime_id,rating,watched_episodes,action,adventure,comedy,drama,sci-fi,space,...,hentai,parody,yuri,yaoi,watching_status_Completed,watching_status_Currently Watching,watching_status_Dropped,watching_status_On Hold,watching_status_Plan to Watch,watching_status_unknown
2280000,7663.0,32949.0,4.0,12.0,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,1,0,0,0,0,0
2280001,7663.0,279.0,5.0,13.0,0.0000,0.0000,0.0000,0.7500,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,1,0,0,0,0,0
2280002,7663.0,690.0,8.0,26.0,0.2500,0.0000,0.0000,0.8750,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,1,0,0,0,0,0
2280003,7663.0,17265.0,7.0,25.0,0.3750,0.2500,0.0000,0.8125,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,1,0,0,0,0,0
2280004,7663.0,23321.0,7.0,25.0,0.5625,0.3750,0.0000,0.7188,0.0000,0.0000,...,0.0,0.0000,0.0,0.0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280373,7663.0,7.0,0.0,0.0,0.8218,0.2226,0.0066,0.0069,0.0005,0.0004,...,0.0,0.0005,0.0,0.0,0,0,0,0,1,0
2280374,7663.0,39617.0,0.0,0.0,0.7327,0.1113,0.0033,0.0034,0.2505,0.0004,...,0.0,0.0005,0.0,0.0,0,0,0,0,1,0
2280375,7663.0,37055.0,0.0,0.0,0.8664,0.0556,0.0016,0.0017,0.1252,0.0004,...,0.0,0.0005,0.0,0.0,0,0,0,0,1,0
2280376,7663.0,25833.0,0.0,0.0,0.7996,0.0278,0.0008,0.0025,0.0626,0.0004,...,0.0,0.0005,0.0,0.0,0,0,0,0,1,0


In [ ]:
raw_chunk["watching_status"] = raw_chunk["watching_status"].apply(lambda x : watching_statusObj[x])

In [ ]:
raw_chunk["watching_status"].value_counts()

In [ ]:
chunk["user_id"].value_counts()

In [ ]:
watching_statusObj

In [ ]:
a = 0.0003
b = 0
def calculateRemaining(primary,isInc):
    remaining = 1 - a if a >= 0.5 else a 
    if (round(primary,3) == 1 or round(primary,3) == 0):
        remaining = - 0.002
    if (isInc == 0):
        result = primary - (remaining / 2)
    else:
        result = primary + (remaining / 2) 
    return round(result, 4)
calculateRemaining(a,b)